In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy
import scipy.io
import numpy as np
from torch.autograd import Variable
import torch.optim as optim

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(520, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        return x #need activation function on x or loss function

In [3]:
# Get input feature dataset
features = scipy.io.loadmat('./feature_vector.mat')

# Turn feature dataset into seperate arrays
AEVs = np.transpose(np.array(features['AEVs']), (2, 0, 1))
Atomic_Num = np.array(features['Atomic_Num'], dtype=np.long)
Target = np.array(features['labels'][0])

print("Shapes:")
print(np.shape(AEVs))
print(np.shape(Atomic_Num))
print(np.shape(Target))

list_atoms = []
for row in Atomic_Num:
    for elem in row:
        list_atoms.append(elem)
print(np.unique(np.array(list_atoms)))

# Seperate into training and testing samples
#train_atoms =    
#train_samples = 
#train_labels = 

#test_atoms = 
#test_samples = 
#test_labels = 

Shapes:
(7165, 23, 520)
(7165, 23)
(7165,)
[0 1 2 3 4]


In [7]:
def doNN():
    # Declare Nets
    NNP_H = Net()
    NNP_C = Net()
    NNP_N = Net()
    NNP_O = Net()
    
    # put in a list for ease of access
    nets = [NNP_H, NNP_C, NNP_N, NNP_O]

    # setting the parameters for the entire net to be zero
    for net in nets:
        net.zero_grad()
    
    # corresponding optimizers
    optimizers = []
    for net in nets:
        optimizers.append( optim.SGD(net.parameters(), lr=0.01, momentum = 0.9) )
    
    # wonder if we need seperate criterions
    criterion = nn.MSELoss()
    
    # Do the thingy
    epochs = 10
    molecules = 200
    
    for epoch in range(epochs) :
        print("epoch: ", epoch)
        outp = []
        for molecule in range(molecules):
            #print("molecule started:")
            for optimizer in optimizers:
                optimizer.zero_grad()
            
            out_f = 0;
            
            for atom in range(23):
                print(Atomic_Num[molecule][atom], end=" "),
                if(Atomic_Num[molecule][atom]==0):
                    continue
                
                aev = torch.from_numpy(AEVs[molecule][atom])
                out = nets[Atomic_Num[molecule][atom]-1](aev.float())
                out_f = out_f + out
            print("")
            
            #target = torch.from_numpy(np.array(Target[molecule])).view(1, -1)
            
            targett = Target[molecule]
            targett = torch.from_numpy(np.array(targett))
            targett = targett.view(1,-1)
            
            #print("asdf")
            #print(targett)
            #out_f = torch.from_numpy(np.array(out_f))
            #print(out_f)
            #out_f = out_f.view(1,-1)
            
            # get list of used atoms i.e NNP's
            used_atoms = []
            for atom in range(23):
                if(Atomic_Num[molecule][atom]!=0):
                    used_atoms.append(Atomic_Num[molecule][atom])
            used_atoms = np.unique(np.array(used_atoms))
            
            if(len(used_atoms)==0) :
                print("scream")
                break
            
            loss = criterion(out_f,targett)

            #print((loss))
            
            loss.backward()
            
            # step only the used optimizers
            
            
            print(used_atoms)
            
            for atom in used_atoms:
                optimizers[atom-1].step()
            
    
doNN()

epoch:  0
2 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2]
2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2]
2 2 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2]
2 2 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2]
2 2 4 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2 4]
2 2 4 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2 4]
2 4 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2 4]
2 2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2]
2 2 2 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2]
2 2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2]
2 2 2 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2]
2 2 3 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2 3]
2 2 3 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2 3]
2 2 3 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2 3]
2 3 2 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2 3]
2 2 4 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2 4]
2 2 2 2 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 
[1 2]
2 2 2 2 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 
[1 2]
2 2 2 2 1 1 1 1 1 1 

KeyboardInterrupt: 